In [1]:
import ee
ee.Initialize()

In [25]:
dem = ee.Image("USGS/GMTED2010")
forestmask = ee.Image('users/marcogirardello/phenoutils/mask_unchanged_500m')
smallgrid = ee.FeatureCollection('users/marcogirardello/phenoutils/grid_export_phenology')

In [7]:
dem1 = dem.updateMask(forestmask.eq(1))

In [8]:
mean_dem = dem1.reduceResolution(ee.Reducer.mean(), bestEffort = False, maxPixels = 900).\
                          reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1) 

In [13]:
mean_slope = ee.Terrain.slope(dem1).reduceResolution(ee.Reducer.mean(), bestEffort = False, maxPixels = 900).\
                          reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1) 

In [15]:
std_slope = ee.Terrain.slope(dem1).reduceResolution(ee.Reducer.stdDev(), bestEffort = False, maxPixels = 900).\
                          reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1) 

In [16]:
std_dem = dem1.reduceResolution(ee.Reducer.stdDev(), bestEffort = False, maxPixels = 900).\
                          reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1)

In [35]:
polyl = list(range(1,42+1))

In [26]:
# Remember year!
for poly in polyl:
    #print(poly)
    # filter a given square
    onesquare = smallgrid.filterMetadata('polyID','equals',poly)
    roi1 = ee.Geometry.Polygon(onesquare.geometry().getInfo().get('coordinates'))
    roi1 = roi1.bounds()
    # polygon number
    polyname = 'mean_dem'+'_'+'poly_'+str(poly)
    # export tile
    filename = 'mean_dem'+'_'+'poly_'+str(poly)
    task = ee.batch.Export.image.toCloudStorage(image = mean_dem,description = polyname,
                                     bucket = 'topography_5km',fileNamePrefix = filename,
                                     scale = 5565.974539663679,fileFormat='GeoTIFF',
                                    skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13,
                                    region = roi1)
    task.start()

NameError: name 'polyl' is not defined

#### Min and Max Elevation

In [34]:
max_dem = dem1.reduceResolution(ee.Reducer.max(), bestEffort = False, maxPixels = 900).\
                          reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1) 
min_dem = dem1.reduceResolution(ee.Reducer.min(), bestEffort = False, maxPixels = 900).\
                          reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1) 
range_dem = max_dem.subtract(min_dem)

In [38]:
range_dem

In [39]:
# Remember year!
for poly in polyl:
    print(poly)
    # filter a given square
    onesquare = smallgrid.filterMetadata('polyID','equals',poly)
    roi1 = ee.Geometry.Polygon(onesquare.geometry().getInfo().get('coordinates'))
    roi1 = roi1.bounds()
    # polygon number
    polyname = 'range_dem'+'_'+'poly_'+str(poly)
    # export tile
    filename = 'range_dem'+'_'+'poly_'+str(poly)
    task = ee.batch.Export.image.toCloudStorage(image = mean_dem,description = polyname,
                                     bucket = 'topography_5km',fileNamePrefix = filename,
                                     scale = 5565.974539663679,fileFormat='GeoTIFF',
                                    skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13,
                                    region = roi1)
    task.start()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [20]:
#Map.addLayer(range,{'min':0,'max':1000,'palette':['blue','green','yellow','red']},'range')

In [22]:
#Map